# Main file for the project 
This is a project mainly contain the implementation of 'attention is all you need' paper.

## Importing the modules
The mostly use module in this project is gonna be numpy for sure so make sure you download it properly before following this notebook.

In [1]:
import pandas as pd # For tabular datasets
import numpy as np # mainly use mathematical library
import torch # Deep learning library 

c:\Users\siwar\.conda\envs\tfpt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Self-attention

### Theory

<h3>Interpretation</h3> 


<div style="display:flex; flex-diretion: row; justify-content:center;;">
    <img src="Image/SA-1.png" width="50%" height="auto" />
</div>

<p>
First, we take a look at the phase "sequence-to-sequence operation". The sequence-to-sequence operation in this case means that when we enter an input of length n to the operation, we expect to get an output of the same length as the input.
After that, we will consider the main operation.
</p>

<div style="display:flex; flex-diretion: row; justify-content:center;;">
    <img src="Image/main-eq-SA.png" width="30%" height="auto" />
</div>


<p>
As you can see there, yi is a sum of the weighted average of each Xj over Xi, multiplied by Xj, which means we get the sum of all information from all inputs that are important to Xi.
Then we got introduced the weighted average operation.
</p>

<div style="display:flex; flex-diretion: row; justify-content:center;;">
    <img src="Image/WA.png" width="30%" height="auto" />
</div>

<p>
the weighted average operation is in fact just a dot product of transposed Xi multiplied by Xj, which tells us the importance of Xj to Xi.
followed by this operation,
</p>
<div style="display:flex; flex-diretion: row; justify-content:center;">
    <img src="Image/WA-softmax.png" width="30%" height="auto" />
</div>

<p>
Instead of using the raw weighted average of xi over xj, we have to make it more polished by mapping the weighted average we got with the softmax operation to scale down our value within the range of [0, 1].
</p>

<div style="display:flex; flex-diretion: row; justify-content:center;">
    <img src="Image/SA-2.png" width="50%" height="auto" />
</div>

<p>
Our diligent work led to the creation of a vector that depicts the entire information of Xi and its relationships to the other inputs.
It's quite fantastic, huh? But based on what I've read, this procedure is also connected to the way our input vectors are built, thus it is safe to assume that the embedded input we received is where the self-attentional trick comes from.
</p>
<div style="display:flex; flex-diretion: row; justify-content:center;">
 <img src="Image/SA-4.png" width="50%" height="auto" />
 <img src="Image/SA-5.png" width="50%" height="auto" />
</div>


### Code

The below code is the basic implementation of self-attention 
<p>
   Before we begin, we learned in the theory section that the algorithm takes input in the form of a sequence of vectors. In practice, however, it is impractical to compute the function in vector terms, so we use a matrix as a representation of a sequence of vectors, with the added benefit of much faster computation.
</p>

<h4>Step</h4>

1. Create inputs sequence matrics 
2. find the raw-weighted-average matrices of given input matrices 
3. softmax the raw-weighted-average matrices
4. Matrix Multiplication of inputs matrices and weighted-average matrics
5. calculate the output sequence matrices

<h4>Utility functions</h4>

1. torch.bmm() -> batched matrix multiplication
2. torch.F.softmax() -> softmax function 

In [16]:
import torch
import torch.nn.functional as F

# assume we have some tensor x with size (b, t, k)
x = torch.tensor(np.random.rand(3,5,5))
print(f'The input: matrices of embedding vectors; {x.shape} dims\n\t {x}')

raw_weights = torch.bmm(x, x.transpose(1, 2))
# - torch.bmm is a batched matrix multiplication. It
#   applies matrix multiplication over batches of
#   matrices.
print(f'The output: raw-weighted-average of all input; {raw_weights.shape} dims \n\t {raw_weights}')

The input: embedding vector; torch.Size([3, 5, 5]) dims
	 tensor([[[0.1063, 0.7249, 0.5281, 0.1914, 0.4143],
         [0.1070, 0.1207, 0.6026, 0.8352, 0.8517],
         [0.1833, 0.2071, 0.8734, 0.5301, 0.7738],
         [0.0499, 0.0671, 0.1472, 0.4868, 0.7437],
         [0.2901, 0.5539, 0.6410, 0.3648, 0.3957]],

        [[0.6603, 0.6671, 0.2745, 0.1479, 0.6052],
         [0.9517, 0.6402, 0.2380, 0.4266, 0.3798],
         [0.2745, 0.1017, 0.5101, 0.3872, 0.9237],
         [0.9911, 0.9060, 0.2101, 0.0605, 0.0012],
         [0.4350, 0.8436, 0.0076, 0.8029, 0.7013]],

        [[0.7694, 0.3651, 0.1221, 0.5660, 0.6281],
         [0.6188, 0.5962, 0.8873, 0.9071, 0.7755],
         [0.0391, 0.1194, 0.0063, 0.3541, 0.7185],
         [0.1152, 0.8517, 0.0768, 0.4013, 0.8173],
         [0.1313, 0.3257, 0.7105, 0.7449, 0.1443]]], dtype=torch.float64)
The output: raw-weighted-average of all input vectors; torch.Size([3, 5, 5]) dims 
	 tensor([[[1.0240, 0.9299, 1.0530, 0.5330, 1.0047],
         [0.92

In [19]:
weights = F.softmax(raw_weights, dim=2)
print(f'weights after softmax; {weights.shape} dims \n\t {weights}')

weights after softmax; torch.Size([3, 5, 5]) dims 
	 tensor([[[0.2206, 0.2008, 0.2271, 0.1350, 0.2164],
         [0.1255, 0.3031, 0.2637, 0.1551, 0.1526],
         [0.1443, 0.2682, 0.2809, 0.1348, 0.1718],
         [0.1464, 0.2692, 0.2301, 0.1948, 0.1594],
         [0.1942, 0.2193, 0.2427, 0.1319, 0.2119]],

        [[0.2076, 0.2224, 0.1479, 0.2038, 0.2184],
         [0.1875, 0.2492, 0.1195, 0.2257, 0.2180],
         [0.1943, 0.1864, 0.2740, 0.1167, 0.2285],
         [0.1864, 0.2451, 0.0813, 0.3151, 0.1721],
         [0.1741, 0.2063, 0.1387, 0.1500, 0.3309]],

        [[0.2401, 0.3400, 0.1159, 0.1769, 0.1271],
         [0.1577, 0.4968, 0.0692, 0.1347, 0.1414],
         [0.1972, 0.2540, 0.1841, 0.2201, 0.1446],
         [0.1792, 0.2941, 0.1310, 0.2738, 0.1219],
         [0.1543, 0.3699, 0.1031, 0.1460, 0.2267]]], dtype=torch.float64)


In [20]:
y = torch.bmm(weights, x) # Matrix's dot product 
print(f'The self-attention matrices; {y.shape} dims \n\t {y}')

The self-attention matrices; torch.Size([3, 5, 5]) dims 
	 tensor([[[0.1561, 0.3601, 0.5945, 0.4750, 0.6242],
         [0.1461, 0.2771, 0.5999, 0.5481, 0.6899],
         [0.1521, 0.2993, 0.6131, 0.5288, 0.6738],
         [0.1425, 0.2876, 0.5714, 0.5279, 0.6760],
         [0.1566, 0.3437, 0.6019, 0.4905, 0.6370]],

        [[0.6863, 0.6647, 0.2298, 0.3705, 0.5001],
         [0.7124, 0.6852, 0.2208, 0.3690, 0.4717],
         [0.5960, 0.5754, 0.2637, 0.4049, 0.6019],
         [0.7658, 0.7202, 0.2185, 0.3209, 0.4021],
         [0.6420, 0.6774, 0.2017, 0.4422, 0.5441]],

        [[0.4367, 0.4963, 0.4356, 0.6510, 0.6607],
         [0.4656, 0.5229, 0.5713, 0.7239, 0.6647],
         [0.3604, 0.4800, 0.3702, 0.6032, 0.6539],
         [0.3725, 0.5293, 0.3913, 0.6153, 0.6762],
         [0.3982, 0.4874, 0.5200, 0.6868, 0.6099]]], dtype=torch.float64)


## Additional tricks